In [1]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
from mesa.space import MultiGrid
import random
import matplotlib.pyplot as plt
import pandas as pd

# Define Buyer and Seller Agents
class BuyerAgent(Agent):
    def __init__(self, unique_id, model, budget):
        super().__init__(unique_id, model)
        self.budget = budget
        self.endowment = 0
        self.learned_price = None
        self.consumer_welfare = 0

    def step(self):
        # Buyer decides to buy if they have enough budget
        if self.budget > 0:
            # Learn only from own past transactions
            price = self.learned_price if self.learned_price else random.uniform(1, 10)
            quantity = random.randint(1, 3)
            self.model.place_order('buy', price, quantity, self)


class SellerAgent(Agent):
    def __init__(self, unique_id, model, endowment):
        super().__init__(unique_id, model)
        self.endowment = endowment
        self.learned_price = None
        self.producer_welfare = 0

    def step(self):
        # Seller decides to sell if they have goods to sell
        if self.endowment > 0:
            # Learn only from own past transactions
            price = self.learned_price if self.learned_price else random.uniform(1, 10)
            quantity = random.randint(1, 3)
            self.model.place_order('sell', price, quantity, self)

# Define the Market Model
class MarketModel(Model):
    def __init__(self, num_buyers, num_sellers, periods):
        self.num_buyers = num_buyers
        self.num_sellers = num_sellers
        self.periods = periods
        self.current_period = 0
        self.running = True
        self.schedule = RandomActivation(self)
        self.order_book = {'buy': [], 'sell': []}
        self.total_welfare = 0
        self.transaction_prices = []
        self.total_consumer_welfare = 0
        self.total_producer_welfare = 0

        # Create buyers and sellers
        for i in range(self.num_buyers):
            buyer = BuyerAgent(i, self, budget=random.uniform(10, 50))
            self.schedule.add(buyer)

        for i in range(self.num_sellers):
            seller = SellerAgent(i + self.num_buyers, self, endowment=random.randint(5, 15))
            self.schedule.add(seller)

        # Data collection
        self.datacollector = DataCollector(
            model_reporters={
                "Average Price": lambda m: m.get_average_transaction_price(),
                "Average Consumer Welfare": lambda m: m.get_average_consumer_welfare(),
                "Average Producer Welfare": lambda m: m.get_average_producer_welfare(),
                "Total Welfare": lambda m: m.total_welfare
            }
        )

    def place_order(self, order_type, price, quantity, agent):
        self.order_book[order_type].append((price, quantity, agent))

    def execute_trades(self):
        # Sort buy orders descending by price and sell orders ascending by price
        buy_orders = sorted(self.order_book['buy'], key=lambda x: -x[0])
        sell_orders = sorted(self.order_book['sell'], key=lambda x: x[0])
        
        while buy_orders and sell_orders:
            buy_price, buy_quantity, buyer = buy_orders[0]
            sell_price, sell_quantity, seller = sell_orders[0]
            
            if buy_price >= sell_price:
                # Determine the quantity to trade
                quantity = min(buy_quantity, sell_quantity)
                transaction_price = (buy_price + sell_price) / 2
                self.transaction_prices.append(transaction_price)
                
                # Update agents
                buyer.budget -= transaction_price * quantity
                seller.endowment -= quantity
                buyer.endowment += quantity
                
                # Agents remember their own transaction price
                buyer.learned_price = transaction_price
                seller.learned_price = transaction_price
                
                # Update welfare
                consumer_surplus = (buy_price - transaction_price) * quantity
                producer_surplus = (transaction_price - sell_price) * quantity
                buyer.consumer_welfare += consumer_surplus
                seller.producer_welfare += producer_surplus
                self.total_consumer_welfare += consumer_surplus
                self.total_producer_welfare += producer_surplus
                self.total_welfare += quantity * (buy_price - sell_price)
                
                # Update order book
                if buy_quantity > quantity:
                    buy_orders[0] = (buy_price, buy_quantity - quantity, buyer)
                else:
                    buy_orders.pop(0)
                
                if sell_quantity > quantity:
                    sell_orders[0] = (sell_price, sell_quantity - quantity, seller)
                else:
                    sell_orders.pop(0)
            else:
                break
        
        # Clear order book for next period
        self.order_book = {'buy': [], 'sell': []}

    def get_average_transaction_price(self):
        return sum(self.transaction_prices) / len(self.transaction_prices) if self.transaction_prices else None

    def get_average_consumer_welfare(self):
        return self.total_consumer_welfare / self.num_buyers if self.num_buyers > 0 else 0

    def get_average_producer_welfare(self):
        return self.total_producer_welfare / self.num_sellers if self.num_sellers > 0 else 0

    def step(self):
        if self.current_period < self.periods:
            self.schedule.step()
            self.execute_trades()
            self.datacollector.collect(self)
            self.current_period += 1
        else:
            self.running = False

# Running the Simulation
num_buyers = 50
num_sellers = 1
periods = 100

market_model = MarketModel(num_buyers, num_sellers, periods)
while market_model.running:
    market_model.step()

# Data Analysis and Visualization
data = market_model.datacollector.get_model_vars_dataframe()
data.to_html('/Users/marcelopisuner/Documents/e) Bodega/_Research training/Advanced data science/ABMs/Mesa Python/Trading games mesa/simulation_output2.html')


Could not import SolaraViz. If you need it, install with 'pip install --pre mesa[viz]'


/var/folders/4x/hgn8xhw16lb6y4fd25yf1trc0000gn/T/ipykernel_78890/1247470239.py:50: DeprecationWarning: The time module and all its Schedulers are deprecated and will be removed in Mesa 3.1. They can be replaced with AgentSet functionality. See the migration guide for details. https://mesa.readthedocs.io/latest/migration_guide.html#time-and-schedulers
  self.schedule = RandomActivation(self)


AttributeError: 'MarketModel' object has no attribute 'random'